In [ ]:
# 目的:
# 先將Label標籤改成 AL=0 MB=1 Central_complex=2
# 再將重複的txt合併輸出同時也將圖片合併複製到新的資料夾

In [1]:
# Step1 修改txt物件標籤編號
import os

def modify_labels(input_label_dir, output_label_dir, target_label):
    os.makedirs(output_label_dir, exist_ok=True)
    
    for label_file in os.listdir(input_label_dir):
        if label_file.endswith('.txt'):
            input_label_path = os.path.join(input_label_dir, label_file)
            output_label_path = os.path.join(output_label_dir, label_file)
            
            with open(input_label_path, 'r') as file:
                lines = file.readlines()
                
            with open(output_label_path, 'w') as file:
                for line in lines:
                    parts = line.split()
                    parts[0] = str(target_label)
                    file.write(' '.join(parts) + '\n')
# 修改AL腦區的標籤為0並輸出到新資料夾
modify_labels('Neuron_image_and_label/正式版_AL_only_txt_Zmax_Zsum', 'Neuron_image_and_label/修改版_AL_only_txt_Zmax_Zsum', 0)

                    
# 修改MB腦區的標籤為1並輸出到新資料夾
modify_labels('Neuron_image_and_label/正式版_MB_only_txt_Zmax_Zsum', 'Neuron_image_and_label/修改版_MB_only_txt_Zmax_Zsum', 1)

# 修改Central_complex腦區的標籤為2並輸出到新資料夾
modify_labels('Neuron_image_and_label/正式版_Central_complex_only_txt_Zmax_Zsum', 'Neuron_image_and_label/修改版_Central_complex_only_txt_Zmax_Zsum', 2)


In [2]:
!rm -r Neuron_image_and_label/正式版_AllinOne_only_image_Zmax_Zsum/
!rm -r Neuron_image_and_label/正式版_AllinOne_only_txt_Zmax_Zsum/

In [4]:
# Step2 合併影像和標籤 ============此版本是將所有圖片來自AL MB CC 都放入 AllinOne 圖片和txt資料夾(總共948張)
# 但是出現一個問題: MB只有660張，導致性能下降
import shutil

def merge_files(image_dirs, label_dirs, output_image_dir, output_label_dir):
    os.makedirs(output_image_dir, exist_ok=True)
    os.makedirs(output_label_dir, exist_ok=True)

    all_images = set()
    all_labels = {}

    # 先處理影像
    for image_dir in image_dirs:
        for image_file in os.listdir(image_dir):
            if image_file.endswith('.png'):  # 或其他影像副檔名
                all_images.add(image_file)
                shutil.copy(os.path.join(image_dir, image_file), os.path.join(output_image_dir, image_file))

    # 再處理標籤
    for label_dir in label_dirs:
        for label_file in os.listdir(label_dir):
            if label_file.endswith('.txt'):
                if label_file not in all_labels:
                    all_labels[label_file] = []
                with open(os.path.join(label_dir, label_file), 'r') as file:
                    all_labels[label_file].extend(file.readlines())

    # 寫入合併後的標籤
    for label_file, lines in all_labels.items():
        with open(os.path.join(output_label_dir, label_file), 'w') as file:
            file.writelines(lines)

# 設定影像和標籤資料夾路徑
image_dirs = ['Neuron_image_and_label/正式版_AL_only_image_Zmax_Zsum', 'Neuron_image_and_label/正式版_MB_only_image_Zmax_Zsum', 'Neuron_image_and_label/正式版_Central_complex_only_image_Zmax_Zsum']
label_dirs = ['Neuron_image_and_label/修改版_AL_only_txt_Zmax_Zsum', 'Neuron_image_and_label/修改版_MB_only_txt_Zmax_Zsum', 'Neuron_image_and_label/修改版_Central_complex_only_txt_Zmax_Zsum']
output_image_dir = 'Neuron_image_and_label/正式版_AllinOne_only_image_Zmax_Zsum'
output_label_dir = 'Neuron_image_and_label/正式版_AllinOne_only_txt_Zmax_Zsum'

merge_files(image_dirs, label_dirs, output_image_dir, output_label_dir)


In [3]:
# Step2 新版合併影像和標籤==========此版本只針對MB資料夾660張進行複製，然而將相同檔案名稱的標籤檔案從AL & CC一起合併至AllinOne
import os
import shutil

def merge_files_based_on_mb(image_src_dir, mb_label_dir, al_label_dir, cc_label_dir, output_image_dir, output_label_dir):
    os.makedirs(output_image_dir, exist_ok=True)
    os.makedirs(output_label_dir, exist_ok=True)

    all_labels = {}

    # 處理影像和MB標籤
    for image_file in os.listdir(image_src_dir):
        if image_file.endswith('.png'):  # 或其他影像副檔名
            mb_label_file = image_file.replace('.png', '.txt')
            shutil.copy(os.path.join(image_src_dir, image_file), os.path.join(output_image_dir, image_file))

            if mb_label_file not in all_labels:
                all_labels[mb_label_file] = []

            # 合併MB標籤
            mb_label_path = os.path.join(mb_label_dir, mb_label_file)
            if os.path.exists(mb_label_path):
                with open(mb_label_path, 'r') as file:
                    all_labels[mb_label_file].extend(file.readlines())

            # 合併AL標籤
            al_label_path = os.path.join(al_label_dir, mb_label_file)
            if os.path.exists(al_label_path):
                with open(al_label_path, 'r') as file:
                    all_labels[mb_label_file].extend(file.readlines())

            # 合併CC標籤
            cc_label_path = os.path.join(cc_label_dir, mb_label_file)
            if os.path.exists(cc_label_path):
                with open(cc_label_path, 'r') as file:
                    all_labels[mb_label_file].extend(file.readlines())

    # 寫入合併後的標籤
    for label_file, lines in all_labels.items():
        with open(os.path.join(output_label_dir, label_file), 'w') as file:
            file.writelines(lines)

# 設定資料夾路徑
image_src_dir = 'Neuron_image_and_label/正式版_MB_only_image_Zmax_Zsum'
mb_label_dir = 'Neuron_image_and_label/修改版_MB_only_txt_Zmax_Zsum'
al_label_dir = 'Neuron_image_and_label/修改版_AL_only_txt_Zmax_Zsum'
cc_label_dir = 'Neuron_image_and_label/修改版_Central_complex_only_txt_Zmax_Zsum'
output_image_dir = 'Neuron_image_and_label/正式版_AllinOne_only_image_Zmax_Zsum'
output_label_dir = 'Neuron_image_and_label/正式版_AllinOne_only_txt_Zmax_Zsum'

merge_files_based_on_mb(image_src_dir, mb_label_dir, al_label_dir, cc_label_dir, output_image_dir, output_label_dir)


# ===================================臨時新增: 因為MB 表現差，所以將 正式版_MB_only_image_Zmax_Zsum資料夾內圖片和 修改版_MB_only_txt_Zmax_Zsum 標註答案修改檔案名稱也複製到正式版_AllinOne_only_image_Zmax_Zsum & 正式版_AllinOne_only_txt_Zmax_Zsum
# 此問題是 948 +660(只有MB標籤，AL&CC都故意不給標籤) 這樣雖然會讓MB上升，但訓練時會造成混淆，因為模型發現有些圖片應該有AL CC卻答案沒有給。
import os
import shutil

def copy_and_rename_files(image_src_dir, label_src_dir, image_dst_dir, label_dst_dir, prefix):
    os.makedirs(image_dst_dir, exist_ok=True)
    os.makedirs(label_dst_dir, exist_ok=True)
    
    # 複製並重新命名影像檔案
    for image_file in os.listdir(image_src_dir):
        if image_file.endswith('.png'):  # 或其他影像副檔名
            new_image_name = prefix + "_" + image_file
            shutil.copy(os.path.join(image_src_dir, image_file), os.path.join(image_dst_dir, new_image_name))
    
    # 複製並重新命名標籤檔案
    for label_file in os.listdir(label_src_dir):
        if label_file.endswith('.txt'):
            new_label_name = prefix + "_" + label_file
            shutil.copy(os.path.join(label_src_dir, label_file), os.path.join(label_dst_dir, new_label_name))

# 設定資料夾路徑
image_src_dir = 'Neuron_image_and_label/正式版_MB_only_image_Zmax_Zsum'
label_src_dir = 'Neuron_image_and_label/修改版_MB_only_txt_Zmax_Zsum'
image_dst_dir = 'Neuron_image_and_label/正式版_AllinOne_only_image_Zmax_Zsum'
label_dst_dir = 'Neuron_image_and_label/正式版_AllinOne_only_txt_Zmax_Zsum'
prefix = 'MB'

copy_and_rename_files(image_src_dir, label_src_dir, image_dst_dir, label_dst_dir, prefix)


In [4]:
# Step3 將 正式版_AllinOne_only_image_Zmax_Zsum 資料夾內的948張圖片，隨機抽取80%作為訓練集複製到 Neuron_image_and_label/images/train
# 同時將對應的Label 複製到 Neuron_image_and_label/labels/train
import numpy as np
# 設定要讀取的txt資料夾路徑
output_txt_path = "Neuron_image_and_label/正式版_AllinOne_only_txt_Zmax_Zsum/"
# 設定要讀取的圖片資料夾路徑
output_image_path = "Neuron_image_and_label/正式版_AllinOne_only_image_Zmax_Zsum/"


# Step3_0 先刪除先前的資料夾
!rm -r Neuron_image_and_label/labels/
!rm -r Neuron_image_and_label/images/
!mkdir 'Neuron_image_and_label/labels/'
!mkdir 'Neuron_image_and_label/images/'
!mkdir 'Neuron_image_and_label/labels/train'
!mkdir 'Neuron_image_and_label/images/train'
!mkdir 'Neuron_image_and_label/labels/val'
!mkdir 'Neuron_image_and_label/images/val'

# Step3_1 隨機打亂946 將 757 個複製到訓練，將 189 個複製到val驗證
import random
import shutil

src_files = np.array(os.listdir(output_txt_path))
a = np.arange(len(src_files))
random.shuffle(a)
src_files = src_files[a]
# 以上完成隨機打亂 948 個檔案
for i in range(len(src_files)):
    if i<528:
        # 複製檔案至訓練資料夾 先複製 YOLO.txt
        shutil.copy(output_txt_path+src_files[i], 'Neuron_image_and_label/labels/train')
        # 複製檔案至訓練資料夾 再複製 圖片
        shutil.copy(output_image_path+src_files[i][:-4]+'.png', 'Neuron_image_and_label/images/train')
    else:
        # 複製檔案至訓練資料夾 先複製 YOLO.txt
        shutil.copy(output_txt_path+src_files[i], 'Neuron_image_and_label/labels/val')
        # 複製檔案至訓練資料夾 再複製 圖片
        shutil.copy(output_image_path+src_files[i][:-4]+'.png', 'Neuron_image_and_label/images/val')

In [9]:
# 目前已經準備好 訓練集和驗證集